In [82]:
from IPython.core.display import display, HTML
from IPython.display import Image, display
from IPython.utils.text import columnize
# display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\plank\AppData\Local\Temp\ipykernel_28104\3263187567.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [83]:
import json
import pickle

import branca.colormap as cm
import folium
import geopandas as gpd
import h3
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
# from pandas.io.json import json_normalize
import numpy as np


import osmnx as ox
import pandas as pd


import seaborn as sns


from branca.colormap import linear


from folium import GeoJson, Map, Marker


from folium.plugins import MarkerCluster
from geojson.feature import *


from matplotlib.pyplot import imshow


from PIL import Image as pilim
from scipy import stats
from shapely import geometry, ops

In [84]:
# Define the place name or address
place_name = "schkeuditz, Germany"

# Fetch the boundaries of the specified place
gdf = ox.geocode_to_gdf(place_name)

# Print the boundaries
print(gdf)

                                            geometry  bbox_north  bbox_south  \
0  MULTIPOLYGON (((12.17243 51.38408, 12.17300 51...   51.461646   51.332554   

   bbox_east  bbox_west   place_id  osm_type  osm_id        lat        lon  \
0  12.354636  12.172433  118631064  relation  444738  51.396351  12.221629   

      class            type  place_rank  importance addresstype        name  \
0  boundary  administrative          16    0.444556        town  Schkeuditz   

                                      display_name  
0  Schkeuditz, Nordsachsen, Saxony, 04435, Germany  


In [85]:
def load_and_prepare_districts(gdf_districts):

    gdf_districts["geom_geojson"] = gdf_districts["geometry"].apply(
        lambda x: geometry.mapping(x))

    gdf_districts["geom_swap"] = gdf_districts["geometry"].map(
        lambda polygon: ops.transform(
            lambda x, y: (y, x), polygon))

    gdf_districts["geom_swap_geojson"] = gdf_districts["geom_swap"].apply(
        lambda x: geometry.mapping(x))

In [86]:
load_and_prepare_districts(gdf)
gdf

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name,geom_geojson,geom_swap,geom_swap_geojson
0,"MULTIPOLYGON (((12.17243 51.38408, 12.17300 51...",51.461646,51.332554,12.354636,12.172433,118631064,relation,444738,51.396351,12.221629,boundary,administrative,16,0.444556,town,Schkeuditz,"Schkeuditz, Nordsachsen, Saxony, 04435, Germany","{'type': 'MultiPolygon', 'coordinates': [(((12...","MULTIPOLYGON (((51.38408 12.17243, 51.38385 12...","{'type': 'MultiPolygon', 'coordinates': [(((51..."


In [87]:
polygon = gdf.iloc[0]["geometry"]
geojson = folium.GeoJson(polygon)

In [88]:
# df_schkeuditz = pd.read_csv(
#     r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\test\schkeuditz-test.csv')
# df_schkeuditz.head()

In [89]:
df_schkeuditz = pd.read_csv(
    # r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\2024-01-08_schkeuditz\locations_2024-01-08_15-17-10.csv')
    r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\2024-01-08_Schkeuditz\locations_2024-01-08_15-01-22.csv')
df_schkeuditz.head()

,deviceId,measurementId,trackId,timestamp [ms],latitude,longitude,speed [m/s],accuracy [m],modalityType,modalityTypeDistance [m],distance [m],modalityTypeTravelTime [ms],travelTime [ms]
0,01cf37aa-dbd9-442c-8619-fee31115b963,2,0,1687598074000,51.395999,12.223138,0.00,3.79,BICYCLE,0.000000,0.000000,0,0
1,01cf37aa-dbd9-442c-8619-fee31115b963,2,0,1687598075000,51.395999,12.223138,0.00,3.79,BICYCLE,0.000000,0.000000,1000,1000
2,01cf37aa-dbd9-442c-8619-fee31115b963,2,0,1687598076000,51.396000,12.223138,0.00,3.79,BICYCLE,0.000111,0.000111,2000,2000
3,01cf37aa-dbd9-442c-8619-fee31115b963,2,0,1687598077000,51.396003,12.223143,0.77,3.79,BICYCLE,0.000592,0.000592,3000,3000
4,01cf37aa-dbd9-442c-8619-fee31115b963,2,0,1687598078000,51.396014,12.223152,1.24,3.79,BICYCLE,0.001966,0.001966,4000,4000


In [90]:
# index each data point into the spatial index of the specified resolution
for res in range(12, 14):
    col_hex_id = "hex_id_{}".format(res)
    col_geom = "geometry_{}".format(res)
    msg_ = "At resolution {} -->  H3 cell id : {} and its geometry: {} "
    print(msg_.format(res, col_hex_id, col_geom))

    df_schkeuditz[col_hex_id] = df_schkeuditz.apply(
        lambda row: h3.geo_to_h3(
            lat=row["latitude"],
            lng=row["longitude"],
            resolution=res),
        axis=1)

    # use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    df_schkeuditz[col_geom] = df_schkeuditz[col_hex_id].apply(
        lambda x: {"type": "Polygon",
                   "coordinates":
                   [h3.h3_to_geo_boundary(
                       h=x, geo_json=True)]
                   }
    )
# transpose for better display
df_schkeuditz.head().T

At resolution 12 -->  H3 cell id : hex_id_12 and its geometry: geometry_12 
At resolution 13 -->  H3 cell id : hex_id_13 and its geometry: geometry_13 


,0,1,2,3,4
deviceId,01cf37aa-dbd9-442c-8619-fee31115b963,01cf37aa-dbd9-442c-8619-fee31115b963,01cf37aa-dbd9-442c-8619-fee31115b963,01cf37aa-dbd9-442c-8619-fee31115b963,01cf37aa-dbd9-442c-8619-fee31115b963
measurementId,2,2,2,2,2
trackId,0,0,0,0,0
timestamp [ms],1687598074000,1687598075000,1687598076000,1687598077000,1687598078000
latitude,51.395999,51.395999,51.396,51.396003,51.396014
longitude,12.223138,12.223138,12.223138,12.223143,12.223152
speed [m/s],0.0,0.0,0.0,0.77,1.24
accuracy [m],3.79,3.79,3.79,3.79,3.79
modalityType,BICYCLE,BICYCLE,BICYCLE,BICYCLE,BICYCLE
modalityTypeDistance [m],0.0,0.0,0.000111,0.000592,0.001966


In [91]:
# Specify the path to the pickle file
file_path = r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\csvs-cyface\test\extracted_messages.pkl'

# Open the pickle file in read mode
with open(file_path, 'rb') as file:
    # Load the contents of the pickle file
    data = pickle.load(file)

# Print the contents of the pickle file
print(data)

{'original_data': {}, 'bag_files':    ros_type                                          meta_file  \
0         2  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...   
1         2  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...   
2         2  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...   
3         2  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...   
5         2  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...   
7         2  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...   

                                  bag_file_reference    location  \
0  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...  schkeuditz   
1  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...  schkeuditz   
2  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...  schkeuditz   
3  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...  schkeuditz   
5  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...  schkeuditz   
7  /mnt/data/sites/r4r-site/scanroot/freiberg/sch...  schkeuditz   

                   record

In [92]:
df_robots_gps = pd.DataFrame(data["bag_messages"])
df_robots_gps.drop(columns=[col for col in df_robots_gps.columns if col not in [
    'latitude', 'longitude', 'timestamp']], inplace=True)
df_robots_gps

,latitude,longitude,timestamp
0,51.396123,12.219667,1698305926074567936
1,51.396124,12.219667,1698305927074356224
2,51.396125,12.219667,1698305928075747328
3,51.396125,12.219667,1698305929073650688
4,51.396125,12.219668,1698305930072520192
...,...,...,...
12338,51.395837,12.221826,1680262916887894357
12339,51.395835,12.221825,1680262917892933679
12340,51.395834,12.221825,1680262918887552142
12341,51.395833,12.221824,1680262919890655002


In [93]:
# index each data point into the spatial index of the specified resolution
for res in range(12, 14):
    col_hex_id = "hex_id_{}".format(res)
    col_geom = "geometry_{}".format(res)
    msg_ = "At resolution {} -->  H3 cell id : {} and its geometry: {} "
    print(msg_.format(res, col_hex_id, col_geom))

    df_robots_gps[col_hex_id] = df_robots_gps.apply(
        lambda row: h3.geo_to_h3(
            lat=row["latitude"],
            lng=row["longitude"],
            resolution=res),
        axis=1)

    # use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    df_robots_gps[col_geom] = df_robots_gps[col_hex_id].apply(
        lambda x: {"type": "Polygon",
                   "coordinates":
                   [h3.h3_to_geo_boundary(
                       h=x, geo_json=True)]
                   }
    )
# transpose for better display
df_robots_gps.head().T

At resolution 12 -->  H3 cell id : hex_id_12 and its geometry: geometry_12 
At resolution 13 -->  H3 cell id : hex_id_13 and its geometry: geometry_13 


,0,1,2,3,4
latitude,51.396123,51.396124,51.396125,51.396125,51.396125
longitude,12.219667,12.219667,12.219667,12.219667,12.219668
timestamp,1698305926074567936,1698305927074356224,1698305928075747328,1698305929073650688,1698305930072520192
hex_id_12,8c1f1a8f19415ff,8c1f1a8f19415ff,8c1f1a8f19415ff,8c1f1a8f19415ff,8c1f1a8f19415ff
geometry_12,"{'type': 'Polygon', 'coordinates': [((12.21944...","{'type': 'Polygon', 'coordinates': [((12.21944...","{'type': 'Polygon', 'coordinates': [((12.21944...","{'type': 'Polygon', 'coordinates': [((12.21944...","{'type': 'Polygon', 'coordinates': [((12.21944..."
hex_id_13,8d1f1a8f19410bf,8d1f1a8f19410bf,8d1f1a8f19410bf,8d1f1a8f19410bf,8d1f1a8f19410bf
geometry_13,"{'type': 'Polygon', 'coordinates': [((12.21964...","{'type': 'Polygon', 'coordinates': [((12.21964...","{'type': 'Polygon', 'coordinates': [((12.21964...","{'type': 'Polygon', 'coordinates': [((12.21964...","{'type': 'Polygon', 'coordinates': [((12.21964..."


In [94]:
def counts_by_hexagon(df, res):
    """Aggregates the number of busstops at hexagon level"""

    col_hex_id = "hex_id_{}".format(res)
    col_geometry = "geometry_{}".format(res)

    # within each group preserve the first geometry and count the ids
    df_aggreg = df.groupby(by=col_hex_id).agg({col_geometry: "first",
                                               "latitude": "count"})

    df_aggreg.reset_index(inplace=True)
    df_aggreg.rename(columns={"latitude": "value"}, inplace=True)

    df_aggreg.sort_values(by="value", ascending=False, inplace=True)
    return df_aggreg

In [95]:
def base_empty_map(location=[51.39992504531213, 12.216827883218217]):
    """Prepares a folium map centered in a central GPS point of Toulouse"""
    m = folium.Map(location=location,
                   zoom_start=14,
                   #  tiles="cartodbpositron",
                   attr='''© <a href="http://www.openstreetmap.org/copyright">
                      OpenStreetMap</a>contributors ©
                      <a href="http://cartodb.com/attributions#basemaps">
                      CartoDB</a>'''
                   )
    return m

In [96]:
df_aggreg_12 = counts_by_hexagon(df=df_schkeuditz, res=12)
print(df_aggreg_12.shape)
df_aggreg_12.head(5)

(2209, 3)


,hex_id_12,geometry_12,value
1263,8c1f1a8f1ad9dff,"{'type': 'Polygon', 'coordinates': [((12.23286...",1964
1032,8c1f1a8f19107ff,"{'type': 'Polygon', 'coordinates': [((12.21270...",733
1409,8c1f1a8f509d7ff,"{'type': 'Polygon', 'coordinates': [((12.19953...",567
867,8c1f1a8f11661ff,"{'type': 'Polygon', 'coordinates': [((12.22092...",548
520,8c1f1a8e2c369ff,"{'type': 'Polygon', 'coordinates': [((12.22040...",443


In [97]:
df_aggreg_13 = counts_by_hexagon(df=df_schkeuditz, res=13)
print(df_aggreg_13.shape)
df_aggreg_13.head(5)

(6778, 3)


,hex_id_13,geometry_13,value
3630,8d1f1a8f1ad9d7f,"{'type': 'Polygon', 'coordinates': [((12.23296...",1954
4073,8d1f1a8f509d6ff,"{'type': 'Polygon', 'coordinates': [((12.19964...",567
2455,8d1f1a8f116613f,"{'type': 'Polygon', 'coordinates': [((12.22092...",519
1356,8d1f1a8e2c3687f,"{'type': 'Polygon', 'coordinates': [((12.22055...",441
2936,8d1f1a8f191073f,"{'type': 'Polygon', 'coordinates': [((12.21270...",413


In [98]:
# make a dictionary of mappings resolution -> dataframes, for future use
dict_aggreg_hex = {12: df_aggreg_12,
                   13: df_aggreg_13}

In [99]:
initial_map = base_empty_map()

In [100]:
df_robots_aggreg_12 = counts_by_hexagon(df=df_robots_gps, res=12)
print(df_robots_aggreg_12.shape)
df_robots_aggreg_12.head(5)

(76, 3)


,hex_id_12,geometry_12,value
54,8c1f1a8f19415ff,"{'type': 'Polygon', 'coordinates': [((12.21944...",1162
41,8c1f1a8e25989ff,"{'type': 'Polygon', 'coordinates': [((12.22156...",627
49,8c1f1a8e259c7ff,"{'type': 'Polygon', 'coordinates': [((12.22130...",564
4,8c1f1a8e2490bff,"{'type': 'Polygon', 'coordinates': [((12.22271...",492
40,8c1f1a8e25987ff,"{'type': 'Polygon', 'coordinates': [((12.22168...",410


In [101]:
df_robots_aggreg_13 = counts_by_hexagon(df=df_robots_gps, res=13)
print(df_robots_aggreg_13.shape)
df_robots_aggreg_13.head(5)

(256, 3)


,hex_id_13,geometry_13,value
172,8d1f1a8f194147f,"{'type': 'Polygon', 'coordinates': [((12.21959...",527
181,8d1f1a8f19417bf,"{'type': 'Polygon', 'coordinates': [((12.21965...",420
155,8d1f1a8e259c0ff,"{'type': 'Polygon', 'coordinates': [((12.22134...",366
14,8d1f1a8e2490bbf,"{'type': 'Polygon', 'coordinates': [((12.22267...",355
159,8d1f1a8e259c67f,"{'type': 'Polygon', 'coordinates': [((12.22145...",336


In [102]:
dict_robots_aggreg_hex = {
    12: df_robots_aggreg_12,
    13: df_robots_aggreg_13
}

## III.2. Visualization with choropleth map

In [103]:
def hexagons_dataframe_to_geojson(df_hex, hex_id_field,
                                  geometry_field, value_field,
                                  file_output=None):
    """Produce the GeoJSON representation containing all geometries in a dataframe
     based on a column in geojson format (geometry_field)"""

    list_features = []

    for i, row in df_hex.iterrows():
        feature = Feature(geometry=row[geometry_field],
                          id=row[hex_id_field],
                          properties={"value": row[value_field]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    geojson_result = json.dumps(feat_collection)

    # optionally write to file
    if file_output is not None:
        with open(file_output, "w") as f:
            json.dump(feat_collection, f)

    return geojson_result


# --------------------------------------------------------------------


def choropleth_map(df_aggreg, hex_id_field, geometry_field, value_field,
                   layer_name, initial_map=None, kind="linear",
                   border_color='None', fill_opacity=0.7,
                   with_legend=False):
    """Plots a choropleth map with folium"""

    # if initial_map is None:
    #     initial_map = base_empty_map()

    # the custom colormap depends on the map kind
    if kind == "linear":
        min_value = df_aggreg[value_field].min()
        max_value = df_aggreg[value_field].max()
        m = round((min_value + max_value) / 2, 0)
        custom_cm = cm.LinearColormap(['green', 'yellow', 'red'],
                                      vmin=min_value,
                                      vmax=max_value)
    elif kind == "outlier":
        # for outliers, values would be -1,0,1
        custom_cm = cm.LinearColormap(['blue', 'white', 'red'],
                                      vmin=-1, vmax=1)
    elif kind == "filled_nulls":
        min_value = df_aggreg[df_aggreg[value_field] > 0][value_field].min()
        max_value = df_aggreg[df_aggreg[value_field] > 0][value_field].max()
        m = round((min_value + max_value) / 2, 0)
        custom_cm = cm.LinearColormap(['silver', 'green', 'yellow', 'red'],
                                      index=[0, min_value, m, max_value],
                                      vmin=min_value,
                                      vmax=max_value)

    # create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_aggreg, hex_id_field,
                                                 geometry_field, value_field)

    return GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity
        },
        name=layer_name
    )

In [104]:
def generate_choropleth_map(df_aggreg_n, resolution, layer_name=None):
    m_hex = choropleth_map(df_aggreg=df_aggreg_n,
                           hex_id_field=f"hex_id_{resolution}",
                           geometry_field=f"geometry_{resolution}",
                           value_field="value",
                           layer_name=layer_name,
                           with_legend=True)
    return m_hex

In [105]:
# geojson = generate_choropleth_map(df_schkeuditz, 13)

In [106]:
# border_color = 'black'
# fill_opacity = 0.7

In [107]:
# plot on map
# feature_group2 = folium.FeatureGroup(name=f'Choropleth 2')

# GeoJson(
#     geojson_data,
#     style_function=lambda feature: {
#         'fillColor': custom_cm(feature['properties']['value']),
#         'color': border_color,
#         'weight': 1,
#         'fillOpacity': fill_opacity
#     },
#     name="Test"
# ).add_to(feature_group2)

# add legend (not recommended if multiple layers)
# if with_legend is True:
#     custom_cm.add_to(initial_map)

In [108]:
feature_group1 = folium.FeatureGroup(name=f'Bikes 13')
df_aggreg_n = counts_by_hexagon(df=df_schkeuditz, res=13)
geojson = generate_choropleth_map(df_aggreg_n, 13, layer_name="Bike 13")
geojson.add_to(feature_group1)

In [109]:
feature_group2 = folium.FeatureGroup(name=f'Robots 13')
df_aggreg_n = counts_by_hexagon(df=df_robots_gps, res=13)
geojson2 = generate_choropleth_map(df_aggreg_n, 13, layer_name="Robot 13")
geojson2.add_to(feature_group2)

In [110]:
df_aggreg_n = counts_by_hexagon(df=df_schkeuditz, res=13)
df_aggreg_n.head()

,hex_id_13,geometry_13,value
3630,8d1f1a8f1ad9d7f,"{'type': 'Polygon', 'coordinates': [((12.23296...",1954
4073,8d1f1a8f509d6ff,"{'type': 'Polygon', 'coordinates': [((12.19964...",567
2455,8d1f1a8f116613f,"{'type': 'Polygon', 'coordinates': [((12.22092...",519
1356,8d1f1a8e2c3687f,"{'type': 'Polygon', 'coordinates': [((12.22055...",441
2936,8d1f1a8f191073f,"{'type': 'Polygon', 'coordinates': [((12.21270...",413


In [111]:
df_aggreg_n = counts_by_hexagon(df=df_robots_gps, res=13)
df_aggreg_n.head()

,hex_id_13,geometry_13,value
172,8d1f1a8f194147f,"{'type': 'Polygon', 'coordinates': [((12.21959...",527
181,8d1f1a8f19417bf,"{'type': 'Polygon', 'coordinates': [((12.21965...",420
155,8d1f1a8e259c0ff,"{'type': 'Polygon', 'coordinates': [((12.22134...",366
14,8d1f1a8e2490bbf,"{'type': 'Polygon', 'coordinates': [((12.22267...",355
159,8d1f1a8e259c67f,"{'type': 'Polygon', 'coordinates': [((12.22145...",336


In [121]:
result_df = pd.merge(counts_by_hexagon(df=df_schkeuditz, res=13), counts_by_hexagon(
    df=df_robots_gps, res=13), on='hex_id_13', how='inner', suffixes=('_df1', '_df2'))
result_df['value'] = result_df['value_df1'] + result_df['value_df2']
result_df.rename(columns={"geometry_13_df1": "geometry_13"}, inplace=True)
result_df.head()

,hex_id_13,geometry_13,value_df1,geometry_13_df2,value_df2,value
0,8d1f1a8f194167f,"{'type': 'Polygon', 'coordinates': [((12.21985...",10,"{'type': 'Polygon', 'coordinates': [((12.21985...",3,13
1,8d1f1a8e24b56bf,"{'type': 'Polygon', 'coordinates': [((12.22246...",3,"{'type': 'Polygon', 'coordinates': [((12.22246...",82,85
2,8d1f1a8e24b0a3f,"{'type': 'Polygon', 'coordinates': [((12.22232...",2,"{'type': 'Polygon', 'coordinates': [((12.22232...",7,9
3,8d1f1a8f194177f,"{'type': 'Polygon', 'coordinates': [((12.21979...",2,"{'type': 'Polygon', 'coordinates': [((12.21979...",115,117
4,8d1f1a8f19413bf,"{'type': 'Polygon', 'coordinates': [((12.21984...",2,"{'type': 'Polygon', 'coordinates': [((12.21984...",3,5


In [122]:
feature_group3 = folium.FeatureGroup(name=f'Both 13')
geojson3 = generate_choropleth_map(result_df, 13, layer_name="B 13")
geojson3.add_to(feature_group3)

In [123]:
feature_group1.add_to(initial_map)
feature_group2.add_to(initial_map)
feature_group3.add_to(initial_map)
folium.LayerControl().add_to(initial_map)
initial_map

In [124]:
# # Second dataset/resolution
# feature_group2 = folium.FeatureGroup(name=f'Choropleth 2')
# hex_map = generate_choropleth_map(df_schkeuditz, 13, feature_group2)
# feature_group2.add_to(m)

In [125]:
# First dataset/resolution
# resolution_1 = "bicycle 13"
# feature_group1 = folium.FeatureGroup(name=f'Choropleth {resolution_1}')
# generate_choropleth_map(df_schkeuditz, 13, feature_group1)
# feature_group1.add_to(m)

In [126]:
# Second dataset/resolution
# resolution_2 = "robots 13"
# feature_group2 = folium.FeatureGroup(name=f'Choropleth {resolution_2}')
# generate_choropleth_map(df_schkeuditz, 13, feature_group2)
# feature_group2.add_to(m)

In [127]:
# Add layer control

In [128]:
# generate_choropleth_map(df_schkeuditz, 13)

In [129]:
generate_choropleth_map(df_robots_gps, 13)

KeyError: 'value'

In [ ]:
def fill_hexagons(geom_geojson, res, flag_swap=False, flag_return_df=False):
    """Fills a geometry given in geojson format with H3 hexagons at specified
    resolution. The flag_reverse_geojson allows to specify whether the geometry
    is lon/lat or swapped"""

    set_hexagons = h3.polyfill(geojson=geom_geojson,
                               res=res,
                               geo_json_conformant=flag_swap)
    list_hexagons_filling = list(set_hexagons)

    if flag_return_df is True:
        # make dataframe
        df_fill_hex = pd.DataFrame({"hex_id": list_hexagons_filling})
        df_fill_hex["value"] = 0
        df_fill_hex['geometry'] = df_fill_hex.hex_id.apply(
            lambda x:
            {"type": "Polygon",
             "coordinates": [
                 h3.h3_to_geo_boundary(h=x,
                                       geo_json=True)
             ]
             })
        assert (df_fill_hex.shape[0] == len(list_hexagons_filling))
        return df_fill_hex
    else:
        return list_hexagons_filling

In [ ]:
geom_to_fill = gdf.iloc[0]["geom_swap_geojson"]

dict_fillings = {}
msg_ = "the subzone was filled with {} hexagons at resolution {}"

for res in range(12, 14):
    # lat/lon in geometry_swap_geojson -> flag_reverse_geojson = False
    df_fill_hex = fill_hexagons(geom_geojson=geom_to_fill,
                                res=res,
                                flag_return_df=True)
    print(msg_.format(df_fill_hex.shape[0], res))

    # add entry in dict_fillings
    dict_fillings[res] = df_fill_hex

# --------------------------
dict_fillings[13].head()

ValueError: Only Polygon GeoJSON supported

In [ ]:
dict_filled_aggreg = {}

for res in range(12, 14):
    col_hex_id = "hex_id_{}".format(res)
    df_outer = pd.merge(left=dict_fillings[res][["hex_id", "geometry"]],
                        right=dict_aggreg_hex[res][[col_hex_id, "value"]],
                        left_on="hex_id",
                        right_on=col_hex_id,
                        how="left")
    df_outer.drop(columns=[col_hex_id], inplace=True)
    df_outer["value"].fillna(value=0, inplace=True)

    # add entry to dict
    dict_filled_aggreg[res] = df_outer

# -----------------------------
dict_filled_aggreg[12].sort_values(by="value", ascending=False).head()

In [ ]:
# res_to_plot = 12
# m_filled_aggreg = choropleth_map(df_aggreg=dict_filled_aggreg[res_to_plot],
#                                  hex_id_field="hex_id",
#                                  value_field="value",
#                                  geometry_field="geometry",
#                                  initial_map=None,
#                                  layer_name="Polyfill aggreg",
#                                  with_legend=True,
#                                  kind="filled_nulls")

# m_filled_aggreg

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(15, 10))

# im1 = pilim.open('images/4_choroplth_multiresol_8_region.png', 'r')
# ax.imshow(np.asarray(im1))
# ax.set_axis_off()

In [ ]:
# for index, row in df_schkeuditz.iterrows():
#     if index == 0:
#         coordinates = [[row['latitude'], row['longitude']]]
#     else:
#         coordinates.append([row['latitude'], row['longitude']])

# line = PolyLine(locations=coordinates, color='blue', weight=2)
# line.add_to(map)
# map